In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:
train_generator = datagen.flow_from_directory(
    r"D:\Development\Dataset\catsvsdogs\train",
    batch_size= 20,
    target_size=(128,128),
    class_mode='binary'
)

In [ ]:
test_generator = datagen.flow_from_directory(
    r"D:\Development\Dataset\catsvsdogs\test",
    batch_size= 20,
    target_size=(128,128),
    class_mode='binary'
)


In [ ]:
def resnet(x,filters, kernel_size=3, stride=1, use_shortcut=True):
    shortcut = x
    # First convolution layer
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Second convolution layer
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)

    if use_shortcut:
    # Adjust shortcut if the number of filters has changed
        shortcut = Conv2D(filters, kernel_size=1, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # Add shortcut to the output of the second convolution layer
    x = Add()([x, shortcut])
    x = ReLU()(x)
    
    return x

In [ ]:
inputs = Input(shape=(128,128,3))

# Starting the model creation
x=Conv2D(32, (3,3),padding='same')(inputs)
x=BatchNormalization()(x)
x=ReLU()(x)


# creating ResNet
x= resnet(x, 64)
x= resnet(x, 128)
# x= resnet(x, 256)
# x= resnet(x, 512)
# x= resnet(x, 1024)
 
x = GlobalAveragePooling2D()(x)
outputs = Dense(1, activation='sigmoid')(x)


model = Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_callback = ModelCheckpoint(
    'model_checkpoint.keras',  # Path to save the model file
    monitor='val_loss',     # Metric to monitor for saving the model
    save_best_only=True,     # Save only the best model based on the monitored metric
    mode='min',             # Mode for the monitored metric (min for loss, max for accuracy)
    verbose=1               # Print messages when saving the model
)

In [ ]:
model.fit(
    test_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    callbacks=[checkpoint_callback]
)